In [ ]:
#load webdriver function from selenium
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
from random import random

# Replace link with the android app needed to extract
link = 'https://play.google.com/store/apps/details?id=com.blackpanther.ninjaarashi'
#Setting up Chrome webdriver Options
chrome_options = webdriver.ChromeOptions()

#creating Chrome webdriver instance with the set chrome_options
driver = webdriver.Chrome(r"C:\Users\Nikhar\SkyDrive\Singapore\NUS\Sem 2\Mining Web Data for Business Insights\chromedriver.exe")

driver.get(link)
Ptitle = driver.find_element_by_class_name('AHFaub').text.replace(':', '')
print(Ptitle)

sleep(1)
driver.find_element_by_xpath('//*[@id="yDmH0d"]/div[@class="QKtxw"]/div[@class="WpDbMd"]//div[@class="XnFhVd"]/div').click()
sleep(1)

# Scroll Down
count = 0
for i in range(1,100):

    SCROLL_PAUSE_TIME = 1

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0," + str(last_height) + ");")

        # Wait to load page
        sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    sleep(SCROLL_PAUSE_TIME)
    driver.execute_script("window.scrollTo(0," + str(last_height - 1400) + ");")
    try:
        show_m = driver.find_element_by_class_name('Ir7OJc')
        count += 1
        show_m.click()
    except:
        continue
    if(count > 49):
        break
    
reviews_df = []

for elem in driver.find_elements_by_class_name('Boieuf'):
    try:
        content = elem.get_attribute('outerHTML')
        soup = BeautifulSoup(content, "html.parser")
        if(soup.find('span', jsname='fbQN7e')):
            
            user_name = soup.find('span', class_='js5pLc').get_text()
            date = soup.find('span',class_='oldIDd').get_text()
            rating = len(soup.find_all('div',class_='vQHuPe bUWb7c'))
            review_helpful = soup.find('div',class_='jUL89d y92BAb').get_text()
            text = soup.find('span', jsname='fbQN7e').get_text()
            if(text == ''):
                text = soup.find('span', jsname='bN97Pc').get_text()
            temp = pd.DataFrame({'App Title':Ptitle,'User Name':user_name,'Date':date,'Rating':rating,'Review Helpful':review_helpful,'Review Text':text},index=[0])
            
            reviews_df.append(temp)
    
    except:
        continue

reviews_df = pd.concat(reviews_df,ignore_index=True)

reviews_df.to_csv(Ptitle+'_reviews_list1.csv', encoding='utf-8')

driver.close()
